In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/home/joaquin/Documents/GitHub/skforecast


In [2]:
from skforecast.utils import get_features_range, check_features_range

In [3]:

# Example data used in the notebook
df = pd.DataFrame()
for i in range(5):
    df[f"feature_{i}"] = np.random.rand(5000)

for i in range(5):
    df[f"cat_feature_{i}"] = np.random.choice(["A", "B", "C"], 5000)

dict_dfs = {'series_1': df, 'series_2': df.iloc[:1000]}

In [4]:
from pprint import pprint
ranges = get_features_range(df)
pprint(ranges)

{'cat_feature_0': {'B', 'C', 'A'},
 'cat_feature_1': {'B', 'C', 'A'},
 'cat_feature_2': {'B', 'C', 'A'},
 'cat_feature_3': {'B', 'C', 'A'},
 'cat_feature_4': {'B', 'C', 'A'},
 'feature_0': (np.float64(6.20623100388018e-05),
               np.float64(0.9995459722186273)),
 'feature_1': (np.float64(0.00025131826789559497),
               np.float64(0.99986486622088)),
 'feature_2': (np.float64(6.954580155149603e-05),
               np.float64(0.9998946070577805)),
 'feature_3': (np.float64(1.8543863598896593e-05),
               np.float64(0.9990430402129542)),
 'feature_4': (np.float64(9.851836958174598e-05),
               np.float64(0.9996280695157298))}


In [5]:
ranges_dict = get_features_range(dict_dfs)
pprint(ranges_dict)


{'series_1': {'cat_feature_0': {'B', 'C', 'A'},
              'cat_feature_1': {'B', 'C', 'A'},
              'cat_feature_2': {'B', 'C', 'A'},
              'cat_feature_3': {'B', 'C', 'A'},
              'cat_feature_4': {'B', 'C', 'A'},
              'feature_0': (np.float64(6.20623100388018e-05),
                            np.float64(0.9995459722186273)),
              'feature_1': (np.float64(0.00025131826789559497),
                            np.float64(0.99986486622088)),
              'feature_2': (np.float64(6.954580155149603e-05),
                            np.float64(0.9998946070577805)),
              'feature_3': (np.float64(1.8543863598896593e-05),
                            np.float64(0.9990430402129542)),
              'feature_4': (np.float64(9.851836958174598e-05),
                            np.float64(0.9996280695157298))},
 'series_2': {'cat_feature_0': {'B', 'C', 'A'},
              'cat_feature_1': {'B', 'C', 'A'},
              'cat_feature_2': {'B', 'C', 'A

In [6]:
new_data_valid = pd.DataFrame({
    'feature_0': [0.1, 0.1, 0.1, 0.1, 0.1],
    'cat_feature_0': ['A', 'B', 'C', 'A', 'B']
})

new_data_invalid = pd.DataFrame({
    'feature_0': [0.1, 0.1, 0.1, 0.1, 10],
    'cat_feature_0': ['A', 'B', 'C', 'A', 'D']
})

new_data_invalid_dict = {'series_1': new_data_invalid, 'series_2': new_data_invalid}

check_features_range(ranges, new_data_valid)
check_features_range(ranges, new_data_invalid)

/home/joaquin/Documents/GitHub/skforecast/skforecast/utils/utils.py:3031: UserWarning: 'feature_0' has one or more values outside the range seen during training [0.00006, 0.99955]. This may affect the accuracy of the predictions.
  warnings.warn(msg)
/home/joaquin/Documents/GitHub/skforecast/skforecast/utils/utils.py:3040: UserWarning: 'cat_feature_0' has values not seen during training. Seen values: {'B', 'C', 'A'}. This may affect the accuracy of the predictions.
  warnings.warn(msg)


In [7]:
check_features_range(ranges_dict, new_data_invalid_dict)

/home/joaquin/Documents/GitHub/skforecast/skforecast/utils/utils.py:3031: UserWarning: 'series_1': 'feature_0' has one or more values outside the range seen during training [0.00006, 0.99955]. This may affect the accuracy of the predictions.
  warnings.warn(msg)
/home/joaquin/Documents/GitHub/skforecast/skforecast/utils/utils.py:3040: UserWarning: 'series_1': 'cat_feature_0' has values not seen during training. Seen values: {'B', 'C', 'A'}. This may affect the accuracy of the predictions.
  warnings.warn(msg)


In [8]:
# Libraries
# ==============================================================================
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from skforecast.datasets import fetch_dataset
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursive
from skforecast.plot import plot_prediction_intervals, set_dark_theme

In [9]:
# Download data
# ==============================================================================
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "datetime"], "header": 0}
)

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data['y']
data = data.sort_index()

# Split train-test
# ==============================================================================
steps = 36
data_train = data[:-steps]
data_test  = data[-steps:]

# Create and fit forecaster
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor       = LGBMRegressor(random_state=123, verbose=-1),
                 lags            = 15,
                 window_features = RollingFeatures(stats=['mean'], window_sizes=10),
                 transformer_y   = None, 
             )

forecaster.fit(y=data_train, store_in_sample_residuals=True)
forecaster

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman
/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (204, 2)


=================== 
ForecasterRecursive 
=================== 
Regressor: LGBMRegressor 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] 
Window features: ['roll_mean_10'] 
Window size: 15 
Series name: y 
Exogenous included: False 
Exogenous names: None 
Transformer for y: None 
Transformer for exog: None 
Weight function included: False 
Differentiation order: None 
Training range: [Timestamp('1991-07-01 00:00:00'), Timestamp('2005-06-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: 
    {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0,
    'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1,
    'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0,
    'n_estimators': 100, 'n_jobs': None, 'num_leaves': 31, 'objective': None,
    'random_state': 123, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0,
    'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': -1} 
fit_kwargs: {} 
Creation date: 2025-08-21 17:49:55 
Last fit date: 2025-08-21 17:49:55 
Skforecast version: 0.17.0 
Python version: 3.12.9 
Forecaster id: None

In [24]:
forecaster.predict(steps=steps, last_window=forecaster.last_window_ *1000, warning_drift=True)

2005-07-01    1.142379
2005-08-01    1.142379
2005-09-01    1.142379
2005-10-01    1.142379
2005-11-01    1.142379
2005-12-01    1.142379
2006-01-01    1.142379
2006-02-01    1.142379
2006-03-01    1.142379
2006-04-01    1.142379
2006-05-01    1.142379
2006-06-01    1.142379
2006-07-01    1.142379
2006-08-01    1.142379
2006-09-01    1.142379
2006-10-01    1.142379
2006-11-01    1.142379
2006-12-01    1.142379
2007-01-01    1.142379
2007-02-01    1.142379
2007-03-01    1.142379
2007-04-01    1.142379
2007-05-01    1.142379
2007-06-01    1.142379
2007-07-01    1.142379
2007-08-01    1.142379
2007-09-01    1.142379
2007-10-01    1.142379
2007-11-01    1.142379
2007-12-01    1.142379
2008-01-01    1.142379
2008-02-01    1.142379
2008-03-01    1.142379
2008-04-01    1.142379
2008-05-01    1.142379
2008-06-01    1.142379
Freq: MS, Name: pred, dtype: float64